# Simple RNN Practice

![alt text](./SimpleRNN.png)

## 1. Settings

### 1) Import Required Libraries

In [1]:
import torch 
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import numpy as np

### 2) Hyperparameter & Data

In [2]:
# Preprocessing string data
# alphabet(0-25), others(26~32), start(33), end(34) -> 35 chars

n_hidden = 50 
lr = 0.01
epochs = 2000

string = "hello pytorch.how long can a rnn cell remember?"# show us your limit!"
chars = "abcdefghijklmnopqrstuvwxyz ?!.,:;01"
char_list = [i for i in chars]
char_len = len(char_list)
n_letters = len(char_list)

### 3) String to One-hot

In [3]:
# String to onehot vector
# a -> [1 0 0 ... 0 0]

def string_to_onehot(string):
    start = np.zeros(shape=len(char_list) ,dtype=int)
    end = np.zeros(shape=len(char_list) ,dtype=int)
    start[-2] = 1
    end[-1] = 1
    for i in string:
        idx = char_list.index(i)
        zero = np.zeros(shape=char_len ,dtype=int)
        zero[idx]=1
        start = np.vstack([start,zero])
    output = np.vstack([start,end])
    return output

### 4) One-hot to Character

In [4]:
# Onehot vector to word
# [1 0 0 ... 0 0] -> a 

def onehot_to_word(onehot_1):
    onehot = torch.Tensor.numpy(onehot_1)
    return char_list[onehot.argmax()]

## 2. RNN class

In [5]:
# RNN with 1 hidden layer

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        self.i2h = nn.Linear(input_size, hidden_size)
        self.h2h = nn.Linear(hidden_size, hidden_size)
        self.i2o = nn.Linear(hidden_size, output_size)
        self.act_fn = nn.Tanh()
    
    def forward(self, input, hidden):
        hidden = self.act_fn(self.i2h(input)+self.h2h(hidden))
        output = self.i2o(hidden)
        return output, hidden

    def init_hidden(self):
        return torch.zeros(1, self.hidden_size)
    
rnn = RNN(n_letters, n_hidden, n_letters)

## 3. Loss function & Optimizer

In [6]:
loss_func = nn.MSELoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr=lr)

## 4. Train

In [7]:
one_hot = torch.from_numpy(string_to_onehot(string)).type_as(torch.FloatTensor())

for i in range(epochs):
    rnn.zero_grad()
    total_loss = 0
    hidden = rnn.init_hidden()

    for j in range(one_hot.size()[0]-1):
        input = one_hot[j:j+1,:]
        output, hidden = rnn.forward(input, hidden)
        target = one_hot[j+1]
        loss = loss_func(output.view(-1),target.view(-1))
        total_loss += loss
        input = output

    total_loss.backward()
    optimizer.step()

    if i % 10 == 0:
        print(total_loss)

tensor(2.3850, grad_fn=<AddBackward0>)
tensor(0.8185, grad_fn=<AddBackward0>)
tensor(0.4609, grad_fn=<AddBackward0>)
tensor(0.2473, grad_fn=<AddBackward0>)
tensor(0.1617, grad_fn=<AddBackward0>)
tensor(0.1049, grad_fn=<AddBackward0>)
tensor(0.0777, grad_fn=<AddBackward0>)
tensor(0.0562, grad_fn=<AddBackward0>)
tensor(0.0428, grad_fn=<AddBackward0>)
tensor(0.0301, grad_fn=<AddBackward0>)
tensor(0.0217, grad_fn=<AddBackward0>)
tensor(0.0362, grad_fn=<AddBackward0>)
tensor(0.0158, grad_fn=<AddBackward0>)
tensor(0.0108, grad_fn=<AddBackward0>)
tensor(0.0225, grad_fn=<AddBackward0>)
tensor(0.0085, grad_fn=<AddBackward0>)
tensor(0.0060, grad_fn=<AddBackward0>)
tensor(0.0047, grad_fn=<AddBackward0>)
tensor(0.0078, grad_fn=<AddBackward0>)
tensor(0.0042, grad_fn=<AddBackward0>)
tensor(0.0033, grad_fn=<AddBackward0>)
tensor(0.0024, grad_fn=<AddBackward0>)
tensor(0.0020, grad_fn=<AddBackward0>)
tensor(0.0016, grad_fn=<AddBackward0>)
tensor(0.0014, grad_fn=<AddBackward0>)
tensor(0.0016, grad_fn=<A

## 5. Test

In [8]:
hidden = rnn.init_hidden()
input = one_hot[0:1,:]

for i in range(len(string)):
    output, hidden = rnn.forward(input, hidden)
    print(onehot_to_word(output.data),end="")
    input = output

hello pytorch.how long can a rnn cell remember?